<a href="https://colab.research.google.com/github/HerrSorokin/DSL/blob/main/Job_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Використовуючи представлення граматика, запропоноване у Job #2 напишіть:

*програму, що генерує функцію FIRST
*програму, що генерує функцію FOLLOW

{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}

  token : (class, value)

  class : int

  value : str

  var : str                 # імєя нетермінала

  definition : list(rule)

  rule : list(var | token)  # права частина правила

To find FIRST function for our grammacy, we are looking through every rule for the non-terminal by next way:
1. If the rule starts with terminal, we add this terminal to the result set and go on to the next rule, if it is.
2. If the rule starts with a non-terminal, we call FIRST function for this non-terminal and analyze the achieved set. If this set contains an empty symbol(this non-terminal is vanishing), we need to go on to the next symbol of the rule, adding this set to the result set. If the set doesn't contain an empty symbol(this non-terminal is not vanishing), we go on to the next rule.

In the end, we'll apply this FIRST function for every non-terminal and find the resulting set for every non-terminal.

In [102]:
def all_firsts(gram, empt):
  vars_firsts = dict() 
  toks = gram['toks']
  vars = gram['vars']

  '''
  Функция First, которая проверяет правила для нетерминала.
  Перебираем все правила для указанного нетерминала(nterm).
  Посимвольно анализируем каждое правило:
  *если символ терминал, то заносим его в множество искомых и переходим к следующему правилу
  *если символ нетерминал, то спускаемся в рекурсию, если ранее не нашли для его first(), 
   если множество первых терминалов анализируемого нетерминал содержит пустой символ, 
   то переходим к следующему символу правила, иначе к следующему правилу
  '''
  def first(nterm):
    toks_set = set() 
    for rule in vars[nterm]:
      for symb in rule:
        if symb not in vars:
          toks_set.add(symb)
          break;
        else:
          if symb not in vars_firsts.keys():
            vars_firsts[symb] = first(symb)
          toks_set = toks_set.union(vars_firsts[symb])
          if(empt not in toks_set):
            break;
    
    return toks_set

  for nterm in vars.keys():
    if nterm not in vars_firsts:
      vars_firsts[nterm] = set(first(nterm))

  return vars_firsts

To find the result of FOLLOW-function for every non-terminal, we find all terminal symbols which can place after non-terminal by analyzing the set of productive rules. When we've got the rule like A → αBβ, we find the set of FIRST(β) and add it to the 'follow' set for non-terminal B. But if we got an empty symbol contained by FIRST(β), we also add the 'follow' set of nonterminal A to the 'follow' set for non-terminal B (FOLLOW(B) += FOLLOW(A)). Further, we go on to the next rule and repeat our actings while the 'follow' set of non-terminal B is growing.

In [110]:
def follow(gram, empt):
  follows = dict() 
  firsts = all_firsts(G, empt)
  toks = gram['toks']
  vars = gram['vars']
  hvar = gram['hvar']

  for nterm in vars:
    follows[nterm] = set()

  #Вспомогательна функция для нахождения множества крайних левых токенов, выводимых из данной последовательности

  def first_in_rule(rule):
    tokens = set()
    for symb in rule:
      if symb in toks: 
        tokens.add(symb)
        return tokens
      else:
        tokens.update(firsts[symb])
        if empt not in firsts[symb]:
          return tokens
    return tokens


  follows[hvar].add('@') # '@' - как конец строки
  it_grows = True
  while it_grows:
    it_grows = False
    for nterm, definition in vars.items():
      for rule in definition:
        while len(rule): #анализируем каждый символ правила и то что находится или может находится справа от него
          top = rule.pop(0)
          if top not in toks:
            was = len(follows[top])
            fts = first_in_rule(rule)
            follows[top].update(fts.difference(empt))
            if empt in fts:
              follows[top].update(follows[nterm])
            now = len(follows[top])
            if not (was == now):
              it_grows = True
  return follows

In [112]:

gr = {
    'toks': set( [
        (0, 'e'), 
        (1, 'v'), 
        (2, 'n'), 
        (3, '+'),
        (4, '*'),
        (5, '('),
        (6, ')')
    ] ),

    'vars': {
        'S' : [['T', 'C']],

        'C' : [[(3, '+'), 'T', 'C'],
               [(0, 'e')]],
             
        'T' : [['F', 'D'],
               [(0, 'e')]],
             
        'D' : [[(4, '*'), 'F', 'D'],
               [(0, 'e')]],
             
        'F' : [[(1, 'v')],
               [(2, 'n')],
               [(5, '('), 'S', (6, ')')]],
    },
    'hvar': 'S'
}

print("Results of FIRST function for every non-terminal: ", all_firsts(gr, (0, 'e')))
print("Results of FOLLOW function for every non-terminal: ", follow(gr, (0, 'e')))






Results of FIRST function for every non-terminal:  {'F': {(5, '('), (1, 'v'), (2, 'n')}, 'T': {(5, '('), (1, 'v'), (0, 'e'), (2, 'n')}, 'C': {(0, 'e'), (3, '+')}, 'S': {(5, '('), (1, 'v'), (0, 'e'), (2, 'n'), (3, '+')}, 'D': {(0, 'e'), (4, '*')}}
Results of FOLLOW function for every non-terminal:  {'S': {(6, ')'), '@'}, 'C': set(), 'T': {(0, 'e'), (3, '+'), '@'}, 'D': set(), 'F': {(0, 'e'), (4, '*'), (3, '+'), '@'}}


Results of FIRST function for every non-terminal:  
{
  
  'F': {(5, '('), (1, 'v'), (2, 'n')}, 
  
  'T': {(5, '('), (1, 'v'), (0, 'e'), (2, 'n')}, 
  
  'C': {(0, 'e'), (3, '+')}, 'S': {(5, '('), (1, 'v'), (0, 'e'), (2, 'n'), (3, '+')}, 
  
  'D': {(0, 'e'), (4, '*')}}

Results of FOLLOW function for every non-terminal:  
{
  
  'S': {(6, ')'), '@'}, 
  
  'C': set(), 
  
  'T': {(0, 'e'), (3, '+'), '@'}, 
  
  'D': set(), 
  
  'F': {(0, 'e'), (4, '*'), (3, '+'), '@'}}